In [1]:
import pandas as pd
import requests
import py2neo
import dotenv
from web3 import Web3
import re
from dotenv import load_dotenv
import os
load_dotenv()

web3 = Web3(Web3.HTTPProvider('https://eth-mainnet.alchemyapi.io/v2/P667s84hD-xZzok8oF7Rhq38ztt7EMCE'));()

()

In [29]:
## grab ultrasoundmoney Twitter data
url = "https://raw.githubusercontent.com/ultrasoundmoney/ens_twitter_accounts/main/ens_twitter.json"
ens_df = pd.read_json(url)
ens_df.head(5)

,id,handle,name,bio,location
0,3200004419,000sourav,Sourav.eth,Blockchain is our best bet against corruption....,127.0.0.1
1,1257842965654810624,007_east,eastpunker.eth,#Bitcoin Founder of Ruotong Music,日本 東京
2,225499192,00Xafre_h,Patrón.ΞTH,None,None
3,2523228612,01RafaelJimenez,rafa.eth,None,None
4,19783137,0666eth,0666.eth | @metakeyblades 11.12,"nft collector, community builder & project adv...",NYC


In [24]:
# helpers 

# figure out if a field contains an ENS
def find_ens(text):
    tokens = text.split(" ")
    for token in tokens:
        matches = re.search(".*\.eth|\.𝚎𝚝𝚑|\.Ξth|\.⟠|dot eth", token)
        if matches is not None:
            match = matches.group(0)
            return(match.lower())
        else:
            return(None)
        
        
# if one row contains multiple ENS, figure out if they're dupes       
def choose_ens(df_row):
    handle_ens = df_row['handle_ens']
    name_ens = df_row['name_ens']
    bio_ens = df_row['bio_ens']
    twitter_id = df_row['id']
    list_aliases = list()
    list_aliases.append(dict(ens_type = 'handle_ens', ens=handle_ens, twitter_id =twitter_id))
    list_aliases.append(dict(ens_type = 'name_ens', ens=name_ens, twitter_id = twitter_id))
    list_aliases.append(dict(ens_type = 'bio_ens', ens=bio_ens, twitter_id = twitter_id))
    list_aliases =  [i for i in list_aliases if i is not None]
    if len(list_aliases) > 0:
        done = set()
        result = list()
        for alias in list_aliases:
            if alias['ens'] not in done:
                if alias['ens'] is not None:
                    done.add(alias['ens'])
                    result.append(alias)
                    return(result)
    else:
        pass


In [25]:
## add potential ens
ens_df['handle_ens'] = ens_df['handle'].apply(find_ens)
ens_df['name_ens'] = ens_df['name'].apply(find_ens)
ens_df['bio_ens'] = ens_df['name'].apply(find_ens)

In [26]:
choose_ens(ens_df.iloc[0])

[{'ens_type': 'name_ens', 'ens': 'sourav.eth', 'twitter_id': 3200004419}]

In [27]:
def create_mapping_df(ens_df):
    mappings_list = list()
    for index, row in ens_df.iterrows():
        ens_options = choose_ens(row)
        if ens_options:
            [mappings_list.append(i) for i in ens_options]
    mapping_df = pd.DataFrame(mappings_list)
    return(mapping_df)


results = create_mapping_df(ens_df)    
        

In [28]:
results.head()

,ens_type,ens,twitter_id
0,name_ens,sourav.eth,3200004419
1,name_ens,eastpunker.eth,1257842965654810624
2,name_ens,rafa.eth,2523228612
3,name_ens,0666.eth,19783137
4,name_ens,mihirg.eth,3119128550


In [29]:
## found matches
results.head(25)
print(len(ens_df) - (len(ens_df) - len(results)))

15861


In [30]:
## missing middle
results['valid_ens'] = results['ens'].apply(web3.ens.is_valid_name)

In [36]:
valid_results = results[results.valid_ens == True]

In [37]:
print(len(valid_results))

15718


In [38]:
valid_results

,ens_type,ens,twitter_id,valid_ens
0,name_ens,sourav.eth,3200004419,True
1,name_ens,eastpunker.eth,1257842965654810624,True
2,name_ens,rafa.eth,2523228612,True
3,name_ens,0666.eth,19783137,True
4,name_ens,mihirg.eth,3119128550,True
...,...,...,...,...
15856,name_ens,zyonik.eth,1450856815772721152,True
15857,name_ens,zywwqq.eth,1145730904041484288,True
15858,name_ens,zyx.eth,1438927646759559168,True
15859,name_ens,ź.eth,1046830729852854272,True


In [42]:
def try_owner(val):
    try:
        response = web3.ens.owner(val)
    except Exception as e:
        response = "DROP"
    return(response)

In [43]:
# add controller wallet
valid_results['owner_wallet'] = results['ens'].apply(try_owner)


/var/folders/6_/msxj91gs3w57w9h1dlk9dgc40000gn/T/ipykernel_83059/1046211245.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_results['owner_wallet'] = results['ens'].apply(try_owner)


In [45]:
valid_results.to_csv('data/wallet_ens_twitter_bliss.csv')

In [6]:
import pandas as pd
valid_results = pd.read_csv('data/wallet_ens_twitter_bliss.csv')

In [8]:
## drop FALSE / unresolved
cleaned_valid = valid_results[valid_results.owner_wallet.isin(['FALSE', '0x0000000000000000000000000000000000000000']) == False]

In [9]:
## drop unresolved
cleaned_valid.to_csv('data/wallet_ens_twitter_cleaned_final.csv')

In [22]:
## join back handle
ens_df_short['twitter_id'] = ens_df['id']
ens_df_short['handle'] = ens_df['handle']

/var/folders/6_/msxj91gs3w57w9h1dlk9dgc40000gn/T/ipykernel_5738/3309925088.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ens_df_short['twitter_id'] = ens_df['id']
/var/folders/6_/msxj91gs3w57w9h1dlk9dgc40000gn/T/ipykernel_5738/3309925088.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ens_df_short['handle'] = ens_df['handle']


In [24]:
cleaned_valid =  cleaned_valid.merge(ens_df_short, on='twitter_id', how='left')

In [26]:
cleaned_valid_final = cleaned_valid.to_csv('data/cleaned_ens_twitter_handles.csv')

In [7]:
print(web3.ens.owner('hydro.eth'))

0xAb48EdD90bdF367d326D827758BAcD2460c59d17
